In [21]:
from pathlib import Path
import pandas as pd
from nvi_survey import create_nvi_survey
from rollups import save_excel_sheets

filepath = (
    Path.cwd() /
    "output" /
    "nvi_2024_analysis_source_2026_2026_20250514.csv"
)
nvi = create_nvi_survey(filepath)

In [ ]:
indicator_groups = (
    nvi.answer_key[~nvi.answer_key["indicator_db_id"].isna()]
    .groupby(["indicator_db_id", "response_type", "tabulate"]) # indicators can only have one response type (possible error though)
)

tables = []

for (indicator, response_type, tabulate), group in indicator_groups:
    if not tabulate:
        continue

    if response_type in {"SINGLE", "GROUPED-SINGLE"}:
        tables.append(
            nvi.compile_single_response_indicator(indicator, "citywide")
            .assign(indicator_db_id=indicator)
        )

    elif response_type == "MULTI-SELECT":
        tables.append(
            nvi.compile_multi_response_indicator(indicator, "citywide")
            .assign(indicator_db_id=indicator)
        )

(
    pd.concat(tables, axis=0)
    .reset_index()
    .assign(indicator_name=lambda df: df["indicator_db_id"]
    .map(nvi.indicator_key)).drop(["citywide", "indicator_db_id"], axis=1)
)[
    ["indicator_name", "count", "universe", "percentage"]
].to_excel("citywide_indicators_20250515.xlsx", index=False)

In [6]:
indicator_groups = (
    nvi.answer_key[~nvi.answer_key["indicator_db_id"].isna()]
    .drop_duplicates(subset=["indicator_db_id", "response_type"]) # indicators can only have one response type (possible error though)
)

result = []
for _, row in indicator_groups.iterrows():
    if not row["tabulate"]:
        continue

    if row["response_type"] in ("SINGLE", "GROUPED-SINGLE"):
        result.append(
            nvi.compile_single_response_indicator(row["indicator_db_id"], "district")
            .assign(indicator=nvi.indicator_key[row["indicator_db_id"]])
        )
    else:
        result.append(
            nvi.compile_multi_response_indicator(row["indicator_db_id"], "district")
            .assign(indicator=nvi.indicator_key[row["indicator_db_id"]])
        )

In [ ]:
(
    pd.concat(result)[["indicator", "count", "universe", "percentage"]]
    .to_excel("nvi_survey_indicator_2024_20250515.xlsx")
)

In [24]:
resident_opportunity_questions = (
    nvi.answer_key[nvi.answer_key["site_category"] == "Resident Opportunity"]
)[["question", "group"]].drop_duplicates()

tables = []
for _, row in resident_opportunity_questions.iterrows():

    tables.append(
        (
            row["question"],
            nvi.tabulate_question(row["question"], row["group"], "citywide")
        )
    )

In [ ]:
save_excel_sheets(tables, Path.cwd() / "output" / "resident_opportunity_breakouts_20250515.xlsx")